In [ ]:
import pandas as pd
import os
import glob
import chardet
import numpy as np
import math
# coding=utf-8

#查詢目錄位置
print( os.getcwd()  )

Folder_Path = "Z:/VPA-13C OAD Data/1_NG"   #要拼接的文件夾及其完整路徑，注意不要包含中文
SaveFile_Path = 'C:/Users/1907075/Project/VAS_Project_Python'    #拼接後要保存的文件路徑
SaveFile_Name = 'all_0720_NG.csv'                                     #合並後要保存的文件名
 

In [2]:
# 判斷編碼型態

def find_encoding(fname):
    r_file = open(fname, 'rb').read() # 'b', meaning 'binary'.
    result = chardet.detect(r_file) # chardet.detect() 函式就可以偵測字串「最有可能」的編碼
    charenc = result['encoding'] #識別編碼
    return charenc

In [10]:
# 資料清理，增加資料
def data_Cleaning(dfname,filename):
    filename= file_list[0]  #取檔名

    dfname.insert(1,'filename' ,filename ) #插入檔名
    #dfname[np.isnan(dfname['Vacuum Data_VG001_Exponent'])]=0
    
    # Vacuum Align System (真空貼合系統)
    # 換算壓力值公式 : log10(Mantissa*10^Exponent)，總共7組 gauge pressure sensor ( Mantissa(尾數) ; Exponent(指數))
    dfname.insert(2,'VG001' ,np.log10(dfname.loc[:,'Vacuum Data_VG001_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG001_Exponent']) )  )  ))   #插入VG001
    dfname.insert(3,'VG002' ,np.log10(dfname['Vacuum Data_VG002_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG002_Exponent']) )  )  ))   #插入VG002
    dfname.insert(4,'VG003' ,np.log10(dfname['Vacuum Data_VG003_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG003_Exponent']) )  )  ))   #插入VG003    
    dfname.insert(5,'VG111' ,np.log10(dfname['Vacuum Data_VG111_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG111_Exponent']) )  )  ))   #插入VG111        
    dfname.insert(6,'VG921' ,np.log10(dfname['Vacuum Data_VG921_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG921_Exponent']) )  )  ))   #插入VG921        
    dfname.insert(7,'VG922' ,np.log10(dfname['Vacuum Data_VG922_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG923_Exponent']) )  )  ))   #插入VG922        
    dfname.insert(8,'VG923' ,np.log10(dfname['Vacuum Data_VG923_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG922_Exponent']) )  )  ))   #插入VG923        
    
    #取代上午/下午
      # df.loc[df.First_name != 'Bill', 'name_match'] = 'Mis-Match'  
    dfname.loc[dfname.DateTime.str.contains("上午") , 'DateTime24'] =  dfname.DateTime.str.replace('上午','') +"AM" 
    dfname.loc[dfname.DateTime.str.contains("下午") , 'DateTime24'] =  dfname.DateTime.str.replace('下午','') +"PM"        

    #轉換 DateTime 24小時制
    dfname.DateTime24 = pd.to_datetime( dfname.DateTime24 ).dt.strftime('%Y-%m-%d %H:%M:%S')
    
     # 移動排序 DateTime24
    cols = list(dfname)
    cols.insert(0,cols.pop(cols.index('DateTime24')))
    dfname = dfname.loc[:,cols]

   # dfname['new_DateTime']=dfname.new_DateTime(1)

    # Row 排序
    #dfname.sort(['DateTime'], ascending=[1])
    dfname.sort_values(by=['filename', 'DateTime24'] )  
    
    #  刪除重複row    
    dfname.drop_duplicates( ['filename', 'DateTime24' ] , keep = 'first' ,inplace=True)   
    
    # 插入rownum
    dfname.insert(1,'rownum' , np.arange(len(dfname)) ) 
    
    return dfname


In [11]:
#修改當前工作目錄
os.chdir(Folder_Path)
#將該文件夾下的所有文件名存入一個列表
file_list = os.listdir()
 
#讀取第一個CSV文件並包含表頭
my_encoding = find_encoding(Folder_Path +'\\'+ file_list[0])
df = pd.read_csv(Folder_Path +'\\'+ file_list[0], encoding=my_encoding)   #編碼默認UTF-8，若亂碼自行更改

#將讀取的第一個CSV文件寫入合並後的文件保存
dfClean=data_Cleaning(df,file_list[0])
dfClean.to_csv(SaveFile_Path+'\\'+ SaveFile_Name,encoding="utf_8",index=False)  # 儲存 Column Name
    
#循環遍歷列表中各個CSV文件名，並追加到合並後的文件
for i in range(1,len(file_list)):
    # 讀檔
    df = pd.read_csv(Folder_Path + '\\'+ file_list[i], encoding=my_encoding)  
    
    # 清洗資料
    dfClean=data_Cleaning(df,file_list[i])
        
   
    # Save All
    dfClean.to_csv(SaveFile_Path+'\\'+ SaveFile_Name,encoding="utf_8",index=False, header=False, mode='a+')  # header=False (不儲存Column Name)

KeyError: 'the label [Vacuum Data_VG001_Mantissa] is not in the [columns]'

In [ ]:
#dfClean

In [ ]:
# load the data using pandas build in read csv function
#df_city_time_series = pd.read_csv('../input/City_time_series.csv',parse_dates=['Date'])
df_city_time_series = dfClean 
# drop null values in ZHVIPerSqft_AllHomes because we are interested in this column
df_city_time_series = df_city_time_series.dropna(subset=['VG001'])
# print the head of our data set
df_city_time_series.head()

In [ ]:
# Time series data source: fpp pacakge in R.
import matplotlib.pyplot as plt
df = pd.read_csv(r'C:\Users\bwich\Google_Cloud\AIA_Project-VAS\Data\all_0720.csv', parse_dates=['rownum'], index_col='rownum')
#df=dfClean
# Draw Plot
def plot_df(df, x, y, title="", xlabel='rownum', ylabel='VG001', dpi=1500):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(df, x=df.index, y=df.VG001, title='VAS Serial.')    

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(20,4), dpi=100)
#pd.read_csv(r'C:\Users\bwich\Google_Cloud\AIA_Project-VAS\Data\all_0720.csv', parse_dates=['rownum'], index_col='rownum').plot(title='Trend Only', legend=False, ax=axes[0])

pd.read_csv(r'C:\Users\bwich\Google_Cloud\AIA_Project-VAS\Data\all_0720.csv', parse_dates=['rownum'], index_col='rownum').plot(title='Seasonality Only', legend=False, ax=axes[1])

#pd.read_csv(r'C:\Users\bwich\Google_Cloud\AIA_Project-VAS\Data\all_0720.csv', parse_dates=['rownum'], index_col='rownum').plot(title='Trend and Seasonality', legend=False, ax=axes

raw_data

In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
cc =  pd.read_csv(r'C:\Users\bwich\Google_Cloud\AIA_Project-VAS\Data\all_0720.csv', parse_dates=['rownum'], index_col='rownum')

In [ ]:
# Data check. 

cc.head()
cc.describe()

In [ ]:
#I observed an conflict in the name 'class'. Therefore, I have changed the name from class to category

#cc= cc.rename(columns={'Class': 'Category'})
cc["Category"]=0

In [ ]:
nor_obs = cc.loc[cc.Category==0]    #Data frame with normal observation
ano_obs = cc.loc[cc.Category==1]    #Data frame with anomalous observation

In [ ]:
# The given dataframe 'cc' is divided into three sets

# Training set: train_features

# Test observations/features: X_test

# Test labels: Y_test

In [ ]:
# 一旦類SVM訓練只有一個類的觀察。 在這種情況下，通過首次200,000次正常交易觀察來訓練算法。 剩下的觀察與異常觀察合併以創建測試集。
# Once class SVM is trained with the observations of only one class. In this case, the algorithm is trained with first 200,000 observation of normal transactions. The remaining observations are merged with the anomalous observation to create a test set.

In [ ]:
# Once class SVM is trained with the observations of only one class. In this case, the algorithm is trained with 
# first 200,000 observation of normal transactions. The remaining observation is merged with the anomalous observation 
# to create a test set. 

train_feature = nor_obs.loc[0:200000, :]
train_feature = train_feature.drop('Category', 1)
Y_1 = nor_obs.loc[200000:, 'Category']
Y_2 = ano_obs['Category']